In [ ]:
import pandas as pd
import seaborn as sns
import os
from sktime.forecasting.base import ForecastingHorizon
from sklearn.neighbors import KNeighborsRegressor
from sktime.utils.plotting import plot_series
from sktime.datasets import load_airline
import matplotlib.pyplot as plt
import matplotlib as mpl
from sktime.forecasting.model_selection import (
    temporal_train_test_split,
)
from sktime.forecasting.compose import (
    EnsembleForecaster,
    make_reduction,
)
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.model_evaluation import evaluate
from tqdm import tqdm
import glob
from pyspark.sql import SparkSession

mpl.use('TkAgg')
%matplotlib inline

In [ ]:
import kcu

kcu.dataset.SandP500.get_dataset()

In [ ]:
# Lets read the data

cwdir = os.getcwd()
# Get list of all stocks:
files = glob.glob(cwdir + "/../../data/sandp500/individual_stocks_5yr/individual_stocks_5yr/*.csv")

spark = SparkSession.builder \
          .appName("SparkByExamples.com") \
          .getOrCreate()

all_stocks = dict()

for file in tqdm(files):
    filename = file[file.rfind("/")+1:-9]
    #df = spark.read.csv(file)
    all_stocks[filename] = spark.read.csv(file, header=True)

print("done.")
#df = spark.read.csv(cwdir + "/../../data/S&P500/individual_stocks_5yr/individual_stocks_5yr/")

In [ ]:
print(all_stocks["JCI"].show(5))

In [ ]:
aapl = all_stocks["AAPL"]
jci = all_stocks["JCI"]
curr = aapl.toPandas()["open"].astype('float64')
#import seaborn as sns
#sns.scatterplot(curr)

fig = plot_series(curr);
#fig.show()
plt.show()
#plot_series(jci.toPandas()["open"]);

In [ ]:
data = curr
y_train, y_test = temporal_train_test_split(data, test_size=300)
plot_series(y_train, y_test, labels=["y_train", "y_test"])
print(y_train.shape[0], y_test.shape[0])
plt.show()

fh = ForecastingHorizon(y_test.index, is_relative=False)
regressor = KNeighborsRegressor(n_neighbors=1)
forecaster = make_reduction(regressor, window_length=15, strategy="recursive")
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
#mean_absolute_percentage_error(y_pred, y_test)
plt.show()
